In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {
    'User-Agent': 'Mozilla/5.0'
}

# Daftar kategori dan tag URL di Detik.com
kategori_list = {
    'Non-Sepak Bola': ['bulutangkis', 'basket', 'voli'],
    'Liga Inggris': ['liga-inggris'],
    'Liga Indonesia': ['liga-1'],
    'Liga Spanyol': ['la-liga'],
    'Liga Italia': ['serie-a']
}

semua_berita = []
nomor = 1

for kategori, tag_list in kategori_list.items():
    for tag in tag_list:
        print(f"\n🔄 Mengambil berita dari tag: {tag} (Kategori: {kategori})")

        halaman = 1
        berita_per_kategori = 0

        while halaman <= 3 and berita_per_kategori < 25:  # max 3 halaman, 25 berita per kategori
            url = f"https://www.detik.com/tag/{tag}?page={halaman}"
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'html.parser')
            links = soup.select("article a")

            for a in links:
                link = a['href']
                try:
                    r_artikel = requests.get(link, headers=headers)
                    soup_artikel = BeautifulSoup(r_artikel.text, 'html.parser')
                    paragraphs = soup_artikel.select('div.detail__body-text p')
                    isi_teks = ' '.join(p.get_text(strip=True) for p in paragraphs)

                    if len(isi_teks.split()) >= 100:
                        semua_berita.append({
                            'No': nomor,
                            'Text': isi_teks,
                            'URL': link,
                            'Kategori': kategori
                        })
                        nomor += 1
                        berita_per_kategori += 1

                        if berita_per_kategori >= 25:
                            break
                    time.sleep(1)
                except Exception as e:
                    print(f"❗ Error mengambil {link}: {e}")
            halaman += 1

print(f"\n✅ Total berita terkumpul: {len(semua_berita)}")

# Simpan ke DataFrame dan ekspor ke CSV
df_final = pd.DataFrame(semua_berita)
df_final.to_csv("berita_olahraga_detik.csv", index=False)
print("\n📁 Data berhasil disimpan ke 'berita_olahraga_detik.csv'")



🔄 Mengambil berita dari tag: bulutangkis (Kategori: Non-Sepak Bola)

🔄 Mengambil berita dari tag: basket (Kategori: Non-Sepak Bola)

🔄 Mengambil berita dari tag: voli (Kategori: Non-Sepak Bola)

🔄 Mengambil berita dari tag: liga-inggris (Kategori: Liga Inggris)

🔄 Mengambil berita dari tag: liga-1 (Kategori: Liga Indonesia)

🔄 Mengambil berita dari tag: la-liga (Kategori: Liga Spanyol)

🔄 Mengambil berita dari tag: serie-a (Kategori: Liga Italia)

✅ Total berita terkumpul: 175

📁 Data berhasil disimpan ke 'berita_olahraga_detik.csv'
